**Импортируем необходимые библиотеки**

In [9]:
import matplotlib.pyplot as plt
import numpy as np
from imageio import imread, imsave
from skimage.color import rgb2gray
import os
import pandas as pd

import skimage.filters 
from skimage.feature import canny

from skimage.transform import hough_line, hough_line_peaks

**Часть с обработкой и сохранением данных**

Здесь происходит выделение фич из изображения с помощью различных методов, которое потом объединяются в датасет. Оба датасета (для обучения и проверки) уже собраны и есть в репозитории. Можно собрать их заново, а можно использовать предоставляемые мной - результат от этого не изменится.

In [10]:
def get_only_vertical(lines):
    """
    Это функция для отбора вертикальных прямых - возвращает из списка прямых только те,
    у которых модуль угла наклона в радианах < 0.1
    """
    angles = list(map(lambda x: x[0], lines))
    lines_arr = np.asarray(lines, dtype=object)
    return lines_arr[np.fabs(angles) < 0.1].tolist()


def get_only_horizontal(lines):
    """
    Это функция для отбора горизонтальных прямых - возвращает из списка прямых только те,
    у которых модуль угла наклона в радианах > 0.9
    """
    angles = list(map(lambda x: x[0], lines))
    lines_arr = np.asarray(lines, dtype=object)
    return lines_arr[np.fabs(angles) > 0.9].tolist()


def max_difference_vertical(vertical_lines, same_angle_th=0.02):
    """
    Находит максимальное расстояние между вертикальными прямыми, которые параллельны (с некоторой точностью)
    """
    sorted_lines = np.array(sorted(vertical_lines))
    max_diff = 0
    for i in range(len(sorted_lines)):
        same_a = same_b = i
        while same_a > 0 and abs(sorted_lines[i][0] - sorted_lines[same_a][0]) < same_angle_th:
            same_a -= 1
        while same_b < len(sorted_lines) - 1 and abs(sorted_lines[i][0] - sorted_lines[same_b][0]) < same_angle_th:
            same_b += 1
        if same_a == same_b:
            continue
        diff = abs(max(sorted_lines[same_a:same_b, 1]) - min(sorted_lines[same_a:same_b, 1]))
        max_diff = max(diff, max_diff)
    return max_diff


def highest_horizontal(horizontal_lines):
    """
    Находит самую верхнюю горзонтальную прямую
    """ 
    if not len(horizontal_lines):
        return 0, 0
    y0 = np.array(list(map(lambda x: x[1]/ np.sin(x[0]), horizontal_lines)))
    min_idx = np.argmin(y0)
    return y0[min_idx], horizontal_lines[min_idx][0]


threshold_methods = [skimage.filters.threshold_otsu,
                     skimage.filters.threshold_yen,
                     skimage.filters.threshold_isodata,
                     skimage.filters.threshold_li,
                     skimage.filters.threshold_mean,
                     skimage.filters.threshold_minimum,
                     skimage.filters.threshold_mean,
                     skimage.filters.threshold_triangle,
                     ]

def find_area_of_box(gray_image ,same_angle_th=0.02):
    """
    Функция для поиска максимальной высоты между параллельными горизонатальными прямыми, угла наклона
    этих кривых и площади темной поверхности, найденной с помощью threshold с максимальным значением
    из всех возвращаемых методами
    """
    results = np.asarray([f(gray_image) for f in threshold_methods])    
    threshold = np.max(results)  
    thresholded_img = gray_image >= threshold
    lines = get_lines_hough(thresholded_img)
    hor_lines = get_only_horizontal(lines)
    sorted_lines = np.array(sorted(hor_lines))
    max_diff = 0
    angle = 0
    for i in range(len(sorted_lines)):
        same_a = same_b = i
        while same_a > 0 and abs(sorted_lines[i][0] - sorted_lines[same_a][0]) < same_angle_th:
            same_a -= 1
        while same_b < len(sorted_lines) - 1 and abs(sorted_lines[i][0] - sorted_lines[same_b][0]) < same_angle_th:
            same_b += 1
        if same_a == same_b:
            continue
        diff = abs(max(sorted_lines[same_a:same_b, 1]) - min(sorted_lines[same_a:same_b, 1]))
        max_diff = max(diff, max_diff)
        if diff == max_diff:
            angle = sorted_lines[i, 0]
    return max_diff, angle, sum((gray_image >= threshold).flatten())

def get_lines_hough(gray_image):
    h, theta, d = hough_line(canny(gray_image))
    _, angle, dist = hough_line_peaks(h, theta, d)
    return sorted(list(zip(angle, dist)))

def get_lines_hough_thresholded(gray_image, threshold):
    return get_lines_hough(gray_image >= threshold)

def get_lines_hough_threshold_yen(gray_image):
    return get_lines_hough_thresholded(gray_image, skimage.filters.threshold_yen(gray_image))

In [11]:
def get_values_for_image(image, isPos):
    """
    Подсчёт всех значений для одного изображения
    """
    gray_image = rgb2gray(image)
    lines = get_lines_hough_threshold_yen(gray_image)
    horizontal_lines = get_only_horizontal(lines)
    vertical_lines = get_only_vertical(lines)
    parallel_max_difference_vert = max_difference_vertical(vertical_lines)
    top_hor_line_pos, top_hor_line_angle = highest_horizontal(horizontal_lines)
    box_height, box_angle, box_area = find_area_of_box(gray_image)
    vert_lines_number = len(vertical_lines)
    hor_lines_number = len(horizontal_lines)
    return (vert_lines_number, hor_lines_number, parallel_max_difference_vert, 
           top_hor_line_pos, top_hor_line_angle, box_height, box_angle, box_area, int(isPos))

In [12]:
columns_names = ['vert_lines', 'hor_lines', 'max_parallel_vert', 'top_hor_line_pos', 
          'top_hor_line_angle', 'box_height', 'box_angle', 'box_area', 'target']


def create_dataset(path_to_images):
    # меняем имена директорий в случае необходимости
    train_pos_dir = os.path.join(path_to_images, 'train/positive/')
    train_neg_dir = os.path.join(path_to_images, 'train/negative/')
    
    test_pos_dir = os.path.join(path_to_images, 'test/positive/')
    test_neg_dir = os.path.join(path_to_images, 'test/negative/')
    
    # соберём информацию для обучения
    # сначала положительные примеры
    train_data = []
    os.chdir(train_pos_dir)
    for image_name in os.listdir(train_pos_dir):
        if image_name != '.DS_Store':
            img = imread(image_name)
            res_tuple = get_values_for_image(img, True)
            train_data.append(res_tuple)
    os.chdir(train_neg_dir)
    # теперь отрицательные примеры
    for image_name in os.listdir(train_neg_dir):
        if image_name != '.DS_Store':
            img = imread(image_name)
            res_tuple = get_values_for_image(img, False)
            train_data.append(res_tuple)
    train_df = pd.DataFrame.from_records(train_data, columns=columns_names)
    
    # соберём информацию для тестирования
    test_data = []
    os.chdir(test_pos_dir)
    for image_name in os.listdir(test_pos_dir):
        if image_name != '.DS_Store':
            img = imread(image_name)
            res_tuple = get_values_for_image(img, True)
            test_data.append(res_tuple)
    os.chdir(test_neg_dir)
    for image_name in os.listdir(test_neg_dir):
        if image_name != '.DS_Store':
            img = imread(image_name)
            res_tuple = get_values_for_image(img, False)
            test_data.append(res_tuple)
    test_df = pd.DataFrame.from_records(test_data, columns=columns_names)
    return train_df, test_df

In [14]:
# путь к данным для обучения - можно изменить для повторного запуска выделения фич
path_to_images = '/Users/evgenia/Desktop/DSET'
train_df, test_df = create_dataset(path_to_images)

In [ ]:
# сохранение датасетов в csv файлы для дальнешего использования

train_df.to_csv('train_df.csv')
test_df.to_csv('test_df.csv')

**Анализ получившихся признаков**

Попробуем посмотреть на корреляцию (линейную) всех признаков с таргетом. Заметим, что особо 'выдающихся' признаков нет, но есть *top_hor_line_angle* с наибольшей по модулю корреляцией с target.

In [16]:
corr = train_df.corr()
corr.style.background_gradient(cmap='coolwarm')

,vert_lines,hor_lines,max_parallel_vert,top_hor_line_pos,top_hor_line_angle,box_height,box_angle,box_area,target
vert_lines,1.000000,0.302578,0.864011,0.434080,0.324403,-0.024444,-0.109623,0.625739,0.175157
hor_lines,0.302578,1.000000,0.102901,0.224526,-0.275817,0.153765,-0.115510,0.036387,0.123257
max_parallel_vert,0.864011,0.102901,1.000000,0.397252,0.233303,-0.045430,-0.089841,0.542555,0.125843
top_hor_line_pos,0.434080,0.224526,0.397252,1.000000,0.149973,0.035321,0.216271,0.454250,0.054915
top_hor_line_angle,0.324403,-0.275817,0.233303,0.149973,1.000000,0.076671,0.121647,0.403363,-0.251686
box_height,-0.024444,0.153765,-0.045430,0.035321,0.076671,1.000000,0.448018,-0.080157,0.083867
box_angle,-0.109623,-0.115510,-0.089841,0.216271,0.121647,0.448018,1.000000,-0.243935,0.076618
box_area,0.625739,0.036387,0.542555,0.454250,0.403363,-0.080157,-0.243935,1.000000,-0.005185
target,0.175157,0.123257,0.125843,0.054915,-0.251686,0.083867,0.076618,-0.005185,1.000000


**Обучение и предсказание классов:**

In [ ]:
path_to_train_dset = './train_df.csv'
path_to_test_dset = './test_df.csv'


train_df = pd.read_csv(path_to_train_dset)
test_df = pd.read_csv(path_to_test_dset)

In [15]:
X_train = train_df.drop(columns=['target'])
y_train = np.ravel(train_df['target'])

X_test = test_df.drop(columns=['target'])
y_test = np.ravel(test_df['target'])

Попробуем предсказывать с помощью двух видов моделей - **дерева решений** и **kNN**.

In [37]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

tree_clf = DecisionTreeClassifier(criterion="entropy", max_depth=4)
tree_clf = tree_clf.fit(X_train,y_train)

In [38]:
y_pred_tree = tree_clf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred_tree))

Accuracy: 0.38461538461538464


Точность, получамая с помощью дерева решений совершенно неудовлетворительна. Данных для применения дерева решений. Попробуем применить kNN. 

In [39]:
from sklearn.neighbors import KNeighborsClassifier

knn_clf = KNeighborsClassifier(n_neighbors=3, weights='distance')

knn_clf.fit(X_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='distance')

In [40]:
y_pred_knn = knn_clf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred_knn))

Accuracy: 0.6153846153846154


Получили не очень высокую точность, но она получилась выше наивного бейслайна. 